# Deploying models using a bring your own container approach (BYOC) using Docker, Azure Container Registries (ACR) and Azure Container Instances (ACI)

Method:
1. Train scikit-learn model
2. Create API and containerise
3. Create container registry and push image
4. Deploy image from ACR to a container instance on ACI

# 1. Train scikit-learn model

In [ ]:
import numpy as np
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import joblib

X_train, y_train, X_test, y_test = [np.load("../data/diabetes.npz")[x] for x in ("X_train", "y_train", "X_test", "y_test")]

alpha=0.1

model = Ridge(alpha=alpha).fit(X_train, y_train)

joblib.dump(model, "diabetes_model.pkl")

# 2. Create API and containerise

This can be infinitely configured. Here, inference is performed on the scikit-learn model using a simple FastAPI REST API which exposes the endpoint on `/`. For example, you could add load balancing by passing traffic through nginx.

In [ ]:
%%writefile requirements.txt

numpy
scikit-learn==0.22.1
joblib
fastapi
uvicorn
pydantic

In [ ]:
%%writefile app.py
from fastapi import FastAPI
import joblib, json
import numpy as np
import sklearn
from pydantic import BaseModel

class InputData(BaseModel):
    data: list

app = FastAPI()

@app.post("/")
async def root(inputData: InputData):
    print(inputData)
    pred = joblib.load('diabetes_model.pkl').predict(np.array(inputData.dict()['data']))
    return {"prediction": pred.tolist()}


In [ ]:
%%writefile Dockerfile
FROM python:3.8
WORKDIR /code
COPY . /code
RUN pip install -r /code/requirements.txt --trusted-host pypi.org --trusted-host files.pythonhosted.org
CMD ["uvicorn","app:app","--host", "0.0.0.0", "--port", "80"]

Docker build your image. You may need to run this command in a local terminal, where Docker is running already, instead of in an Azure ML Compute. 

`docker build -t andreww_byoc:0.1 .`

# 3. Create container registry and push image

We need to setup the Azure CLI to get started. First update the CLI and login. Make sure you are using an account under the correct subscription (change using `--tenant <TENANT_ID>` option).

`sudo apt-get install --only-upgrade azure-cli` (or using `brew` for [mac](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-macos))

`az login --tenant f2ec3ef9-cca6-46ec-be61-845d74fcae94`

Create ACR (Azure container registry) under appropriate resource group and login to it (see Microsoft tutorial in references below for more details)

`az acr create --resource-group data-ai-academy-sept-2022 --name andrewwbyocacr --sku Basic`

`az acr login --name andrewwbyocacr`

Get full name of registry login server

`az acr show --name andrewwbyocacr --query loginServer --output table`

Tag Docker image with registry details

`docker tag andreww_byoc andrewwbyocacr.azurecr.io/andreww_byoc:v1`

Docker push to the ACR

`docker push andrewwbyocacr.azurecr.io/andreww_byoc:v1`

# 4. Deploy image from ACR to a container instance on ACI

You could do this with `az container create` as per the [Microsoft tutorials](https://learn.microsoft.com/en-us/azure/container-instances/container-instances-tutorial-deploy-app), but this requires configuring credentials of the login account. Much easier to do from the Azure Portal browser interface under [Container Instances](https://portal.azure.com/#view/HubsExtension/BrowseResource/resourceType/Microsoft.ContainerInstance%2FcontainerGroups). Click create, and set:

- Resource group: same as one set for ACR
- Container name: new unique name
- Region
- Image source: Azure Container Registry
- Registry: one you created in ACR
- Image + Image tag: image pushed to registry with `docker push`
- Optional: set DNS name label under "Networking" for custom URL

# Test endpoint and delete

Find the endpoint url by looking at the overview of the created ACI in Azure portal.

In [ ]:
endpoint_url = "http://20.26.14.47/"

In [ ]:
import requests
import json
input_payload = json.dumps({
    'data': X_test[0:2].tolist(),
})

requests.post(endpoint_url, input_payload, headers={'Content-Type':'application/json'}).json()

Remember to stop or delete the created ACI in Azure portal!

In [1]:
!rm app.py
!rm diabetes_model.pkl
!rm Dockerfile
!rm requirements.txt

# Manage

# References

https://learn.microsoft.com/en-us/azure/container-instances/container-instances-tutorial-prepare-acr